In [2]:
%%writefile modular/models/baseline_model.py
"""
Defines a PyTorch baseline model for multi-class classification.
"""
import torch
from torch import nn

class BaseLine(nn.Module):
    def __init__(self, input_channels: int, input_height: int, input_width: int, hidden_units: int, output_shape: int) -> None:
        """Defines a simple feedforward neural network for multi-class classification.

        Args:
            input_channels (int): Number of input channels (e.g., 3 for RGB).
            input_height (int): Height of the input images.
            input_width (int): Width of the input images.
            hidden_units (int): Number of hidden units between layers.
            output_shape (int): Number of output units.
        """
        super().__init__()
        input_shape = input_channels * input_height * input_width
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape),
            nn.ReLU()
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layer_stack(x)

Overwriting modular/models/baseline_model.py


In [3]:
%%writefile modular/models/TinyVGG_model.py
"""
Defines a PyTorch model that replicates the TinyVGG architecture.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
    def __init__(self, input_channels: int, input_height: int, input_width: int, hidden_units: int, output_shape: int) -> None:
        """Defines a TinyVGG model.

        Args:
            input_channels (int): Number of input channels (e.g., 3 for RGB).
            input_height (int): Height of the input images.
            input_width (int): Width of the input images.
            hidden_units (int): Number of hidden units between layers.
            output_shape (int): Number of output units.
        """
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, 
                      hidden_units, 
                      kernel_size=3, 
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, 
                      hidden_units, 
                      kernel_size=3, 
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Calculate the size of the flattened features after the convolutional layers
        self._calculate_flattened_size(input_height, input_width, hidden_units)
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=self.flattened_size, out_features=output_shape)
        )
        
    def _calculate_flattened_size(self, height, width, hidden_units):
        # Simulate passing a dummy tensor through the conv layers to get the output size
        dummy_input = torch.zeros(1, hidden_units, height, width)
        dummy_output = self.conv_block_2(self.conv_block_1(dummy_input))
        self.flattened_size = dummy_output.numel()
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

Overwriting modular/models/TinyVGG_model.py


In [1]:
%%writefile modular/models/vgg19_model.py
"""
Defines a PyTorch model that replicates the VGG19 architecture.
"""
import torch
from torch import nn
import torchvision.models as models

class VGG19(nn.Module):
    def __init__(self, output_shape: int, device=None) -> None:
        """Defines a VGG19 model using transfer learning.

        Args:
            output_shape (int): The output shape of the model.
            device (_type_, optional): The device to run the model on. Defaults to None.
        """
        super().__init__()
        self.weights = models.VGG19_Weights.DEFAULT
        self.model = models.vgg19(weights=self.weights).to(device)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Unfreeze the last 7 layers
        for param in self.model.features[-7:].parameters():
            param.requires_grad = True
            
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, output_shape)
        ).to(device)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

Overwriting modular/models/vgg19_model.py


In [2]:
%%writefile modular/models/efficientnet_b3_model.py
"""
Defines a PyTorch model that replicates the EfficientNet B3 architecture.
"""
import torch
from torch import nn
import torchvision.models as models

class EfficientNetB3(nn.Module):
    def __init__(self, output_shape: int, device=None) -> None:
        """Defines an EfficientNet B3 model using transfer learning.

        Args:
            output_shape (int): The output shape of the model.
            device (_type_, optional): The device to run the model on. Defaults to None.
        """
        super().__init__()
        self.weights = models.EfficientNet_B3_Weights.DEFAULT
        self.model = models.efficientnet_b3(weights=self.weights).to(device)
        
        for param in self.model.parameters():
            param.requires_grad = False

        # Unfreeze the last 7 layers
        for param in self.model.features[-7:].parameters():
            param.requires_grad = True
        
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, output_shape)
        ).to(device)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

Overwriting modular/models/efficientnet_b3_model.py


In [ ]:
%%writefile modular/models/mobilenetv2_model.py
